# lens

> A module to model and calculate the properties of lenses

In [ ]:
#| default_exp lens

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from LensCalcPy.parameters import *
from LensCalcPy.utils import *

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import nquad, dblquad
import pickle
from abc import ABC, abstractmethod

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

class Lens():
    """Abstract class for lens objects.
    """
    def __init__(self, mass, u_t = 1, ds=770):
        self.ut_interp = ut_interp
        self.mass = mass
        self.u_t = u_t
        self.ds = ds
        pass

    def __str__(self):
        pass

    __repr__ = __str__

    def umin_lower_bound(self, d):
        return 0

    def umin_upper_bound(self, d):
        pass

    def differential_rate_integrand(self, umin, d, t, dist_func, density_func, v_disp_func, finite=False, density_func_uses_d=False):
        r = dist_func(d)
        ut = self.umin_upper_bound(d) if (self.ut_interp and finite) else self.u_t
        if ut <= umin:
            return 0
        v_rad = velocity_radial(d, self.ds, self.mass, umin, t * htosec, ut)
        v_disp = v_disp_func(r)
        density_input = d if density_func_uses_d else r
        return 2 * (1 / (ut**2 - umin**2)**0.5 *
                density_func(density_input) / (self.mass * v_disp**2) *
                v_rad**4 * (htosec / kpctokm)**2 *
                np.exp(-(v_rad**2 / v_disp**2)))
    
    def differential_rate(self, t, integrand_func, finite=False):
        if finite:
            result, error = dblquad(integrand_func, 0, self.ds, self.umin_lower_bound, self.umin_upper_bound, args=[t], epsabs=0, epsrel=1e-1)
            return result    
        else:
            umin_bounds = [0, self.u_t]
            d_bounds = [0, self.ds]
            result, error = nquad(integrand_func, [umin_bounds, d_bounds], args=[t])
            return result

    # @abstractmethod
    def compute_differential_rate(self, t, finite=False):
        pass


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()